In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from utils import (
    obtener_escalar_de_inflacion,
    show_plot,
)

In [ ]:
def set_bias(last_check: datetime) -> int:
    current_date = datetime.now()
    years_diff = current_date.year - last_check.year
    months_diff = current_date.month - last_check.month
    return years_diff * 12 + months_diff

In [ ]:
last_check = datetime(
    2025, 7, 1
)  # ACTUALIZAR CUANDO SE AGREGUEN DATOS FUTUROS A ESTA FECHA Y REVISAR TODOS LOS DATOS
MONTHS_TO_UPDATE = set_bias(last_check)
MONTHS_TO_UPDATE

In [ ]:
datos_personales = pd.read_csv("../data/sueldos.csv")

In [ ]:
datos_personales["sueldo_bruto_usd_oficial"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_bruto_usd_blue"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_blue"]
)
datos_personales["sueldo_neto_usd_oficial"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_neto_usd_blue"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_blue"]
)
datos_personales.tail()

In [ ]:
sysarmy = pd.read_csv("../data/2025_2_sysarmy.csv")
sysarmy = sysarmy[
    [
        "dedicacion",
        "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos",
        "seniority",
    ]
]
sysarmy["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] = sysarmy[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
] * obtener_escalar_de_inflacion(MONTHS_TO_UPDATE+1)
sysarmy_fulltime = sysarmy[sysarmy["dedicacion"] == "Full-Time"]
promedios_seniority = sysarmy_fulltime.groupby("seniority")[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].mean()
sysarmy_fulltime = sysarmy[
    (sysarmy["dedicacion"] == "Full-Time")
    & (sysarmy["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].notnull())
]
sueldo_actual = pd.DataFrame(
    {
        "dedicacion": "Full-Time",
        "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos": datos_personales[
            "sueldo_neto_ars"
        ].iloc[-1],
        "seniority": "Sueldo Actual",
        "Color": "purple",
        "Ancho": 20,
    },
    index=[0],
)
sysarmy_fulltime = sysarmy_fulltime.sort_values(
    by="ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
).reset_index(drop=True)

juniors = sysarmy_fulltime[sysarmy_fulltime["seniority"] == "Junior"]
semiseniors = sysarmy_fulltime[sysarmy_fulltime["seniority"] == "Semi-Senior"]
seniors = sysarmy_fulltime[sysarmy_fulltime["seniority"] == "Senior"]

juniors_90 = juniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.95)
juniors = juniors[
    juniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] <= juniors_90
]

semiseniors_90 = semiseniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.95)
semiseniors_05 = semiseniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.05)
semiseniors = semiseniors[
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"]
    <= semiseniors_90
]
semiseniors = semiseniors[
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"]
    >= semiseniors_05
]

seniors_90 = seniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.95)
seniors_05 = seniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.05)
seniors = seniors[
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] <= seniors_90
]
seniors = seniors[
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] >= seniors_05
]

In [ ]:
juniors["Color"] = "green"
juniors["Ancho"] = 0.3
semiseniors["Color"] = "blue"
semiseniors["Ancho"] = 0.3
seniors["Color"] = "red"
seniors["Ancho"] = 0.3

sysarmy_ordenado = (
    pd.concat([juniors, semiseniors, seniors, sueldo_actual])
    .sort_values(by="ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos")
    .reset_index(drop=True)
)

unique_seniorities = sysarmy_ordenado["seniority"].unique()
palette = sns.color_palette("husl", len(unique_seniorities))
color_map = dict(zip(unique_seniorities, palette))

plot_id = plt.figure(figsize=(15, 8))
plt.bar(
    range(len(sysarmy_ordenado)),
    sysarmy_ordenado["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"],
    color=sysarmy_ordenado["Color"],
    width=sysarmy_ordenado["Ancho"],
)

plt.bar(0, 0, color="green", label="Junior")
plt.bar(0, 0, color="blue", label="Semi-Senior")
plt.bar(0, 0, color="red", label="Senior")
plt.bar(0, 0, color="purple", label="Sueldo Actual")

show_plot(
    plot_id,
    "Índice de cada salario (ordenado)",
    "Sueldo Neto (ARS)",
    "Distribución de Sueldos IT Full-Time Ordenados por Valor",
)

In [ ]:
semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].mean()

In [ ]:
semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].median()

In [ ]:
summary = [
    sueldo_actual["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"][0],
    juniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].mean(),
    juniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].median(),
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].mean(),
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].median(),
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].mean(),
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].median(),
]

etiquetas = [
    "Mi Neto",
    "Media Jrs",
    "Mediana Jrs",
    "Media SSrs",
    "Mediana SSrs",
    "Media Srs",
    "Mediana Srs",
]

combined = list(zip(summary, etiquetas))
combined.sort(key=lambda x: x[0])
summary, etiquetas = zip(*combined)

color = ["#FDD", "#FBB", "#F99", "#F77", "#F55", "#F33", "#F11"]

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(
    etiquetas,
    summary,
    color=[
        color[index] if value != "Mi Neto" else "purple"
        for index, value in enumerate(etiquetas)
    ],
    width=0.9,
)

for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'${height:,.0f}',
        ha='center',
        va='bottom',
    )

show_plot(
    fig,
    "Categorías",
    "Sueldo en Millones",
    "Comparación del Sueldo Actual Neto con la Media y Mediana de todos los Seniorities",
)